In [1]:
import numpy  as np
import pandas as pd

In [2]:
data    = np.array(pd.read_csv('dataset_61_iris.csv'))
X_train = np.array(data[:, 0:4])
X_train = np.vstack((np.array([1,1,1,1]), X_train))
Y_train = np.array(data[:, 4])

In [31]:
class Node:
    def __init__ (self):
        self.leaf         = False
        self.attr         = None
        self.datatype     = None
        self.split_crit   = None
        self.children     = []
        self.result       = None
        
    '''
     In:  Labelcounts in np.array
     Out: Entropy (0s and 1s respectively)
    '''
    def _entropy(self, label_counts):
        tmp  = label_counts/np.sum(label_counts)
        etrp = tmp * np.log2(tmp, out = np.zeros_like(tmp), where = (label_counts != 0))

        return - np.sum(etrp)

    '''
     In:  Crosstab of X_train and Y_train
     Out: The information gain IG(A,Y) = E(Y) - SUM (|Di| / |D| * E(Di))
    '''
    def _informationGain(self, crosstab):
        etrp_before = self._entropy(np.bincount(np.sum(crosstab, axis=0)))
        etrp_after  = sum([np.sum(row) * self._entropy(row) / np.sum(crosstab) for row in crosstab])

        return etrp_before - etrp_after

    '''
    In:  Data
    Out: 1. Index of attribute to split by
         2. Threshold to split by if found attribute is metric
         3. Classes if found attribute is ordinal
    '''
    def _find_attr(self, X_train, Y_train):
        ig = [] # List of Information Gain Values for each Attribute
        sp = [] # List of Thresholds to split by if Attribute is metric

        for attribute in X_train.T:
            attr = attribute[1:]  # First row always contains Information about the Attributes themselves, no data

            if attribute[0] == 1: # Attribute is metric
                thresholds       = np.convolve(np.sort(attr), np.array([.5, .5]), mode = 'valid')
                informationGains = [self._informationGain(np.array(pd.crosstab([attr > thresholds[i]], Y_train)))
                                    for i in range(0,(len(attr)-1))]

                ig.append(max(informationGains))
                sp.append(thresholds[informationGains.index(max(informationGains))])

            if attribute[0] == 2: # Attribute is ordinal
                ig.append(self._informationGain(np.array(pd.crosstab(attr, Y_train))))
                sp.append(np.unique(attr))   # No Threshold can be calculated

        return ig.index(max(ig)), sp[ig.index(max(ig))]
    
    '''
    In:  Number of children
    Out: List of nodes (children)
    '''
    def _make_children(self, number):
        return [Node() for _ in range(number)]
        
    '''
    fit the model
    '''
    def fit(self, X_train, Y_train):
        
        X_data         = X_train[1:, :]
        unique, counts = np.unique(Y_train, return_counts=True)
        self.result    = dict(zip(unique, counts))        
        print(self.result)
        
        
        if len(self.result) == 1:
            self.leaf = True
        
        else:
            self.attr, self.split_crit = self._find_attr(X_train, Y_train)
            self.datatype              = X_train[0, self.attr]
            
            if self.datatype == 1:

                # only two Nodes, one for each side of the best split we found
                self.children = self._make_children(2)
                indices_left  = X_data[:, self.attr] > self.split_crit  # left split, w/o first row
                indices_right = X_data[:, self.attr] <= self.split_crit # right split, w/o first row
                
                # concatenate the Information about the datatype from the first row
                self.children[0].fit(np.vstack((X_train[0, :], X_data[indices_left, :])), Y_train[indices_left])
                self.children[1].fit(np.vstack((X_train[0, :], X_data[indices_right, :])), Y_train[indices_right])
                
           
            if self.datatype == 2:
                # one child for every unique label in attr
                self.childen = self._make_children(len(self.split_crit)) # Node for every class
                
                # Iterate over all Nodes in children and call fit with data corresponding to the class label in attr
                for Node, clss in zip(self.children, self.split_crit):
                    Node.fit(np.vstack((X_train[0, :], X_data[self.attr == clss, :])), Y_train[self.attr == clss])
            
    '''
    predict label for instance
    '''
    def predict(self, X):
        
        if self.leaf:
            print(self.result)
        
        else:
            
            if self.datatype == 1:
                if X[self.attr] > self.split_crit:
                    self.children[0].predict(X)
                else:
                    self.children[1].predict(X)
                    
            if self.datatype == 2:
                self.children[split_crit.index(X)].predict(X)

In [33]:
tree = Node()
tree.fit(X_train, Y_train)

{'Iris-setosa': 50, 'Iris-versicolor': 50, 'Iris-virginica': 50}
{'Iris-versicolor': 50, 'Iris-virginica': 50}
{'Iris-versicolor': 1, 'Iris-virginica': 45}
{'Iris-virginica': 43}
{'Iris-versicolor': 1, 'Iris-virginica': 2}
{'Iris-virginica': 2}
{'Iris-versicolor': 1}
{'Iris-versicolor': 49, 'Iris-virginica': 5}
{'Iris-versicolor': 2, 'Iris-virginica': 4}
{'Iris-versicolor': 2, 'Iris-virginica': 1}
{'Iris-virginica': 1}
{'Iris-versicolor': 2}
{'Iris-virginica': 3}
{'Iris-versicolor': 47, 'Iris-virginica': 1}
{'Iris-virginica': 1}
{'Iris-versicolor': 47}
{'Iris-setosa': 50}


In [36]:
# Index of training instance in X_train
example = 120

# prediction
tree.predict(X_train[example, :])

# ground truth
print(Y_train[example])

{'Iris-virginica': 3}
Iris-virginica
